In [1]:
from keras.preprocessing.image import load_img, img_to_array

style_path = '/content/tumblr_inline_oh8q2htcex1si5ncs_1280.jfif'

content_path = '/content/tumblr_pr1c6zASx51snegd3o1_500.jpg'

width, height = load_img(content_path).size

img_height = 400

img_width = int(width * img_height / height)

Using TensorFlow backend.


In [0]:
import numpy as np
from keras.applications import vgg19

def preprocess_img(image_path):
  image = load_img(image_path, target_size=(img_height, img_width))
  image = img_to_array(image)
  image = np.expand_dims(image, axis = 0)
  image = vgg19.preprocess_input(image)
  return image

In [0]:
def deprocess_image(x):
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [4]:
from keras import backend as K

content_image = K.constant(preprocess_img(content_path))
style_image = K.constant(preprocess_img(style_path))
generated_image = K.placeholder((1,img_height, img_width, 3))

input_tensor = K.concatenate([content_image, style_image, generated_image], axis=0)
model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet',include_top=False)
print('Model loaded.')







Model loaded.


In [0]:
def content_loss(base, combination):
  return K.sum(K.square(combination - base))

In [0]:
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  return gram

In [0]:
def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [0]:
def total_variation_loss(x):
  a = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, 1:, :img_width - 1, :])
  b = K.square(x[:, :img_height - 1, :img_width - 1, :] - x[:, :img_height - 1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))

In [0]:
output_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

In [10]:
loss = K.variable(0.)
layer_features = output_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

In [0]:
for layer_name in style_layers:
  layer_features = output_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss += (style_weight / len(style_layers)) * sl
  loss += total_variation_weight * total_variation_loss(generated_image)

In [12]:
grads = K.gradients(loss, generated_image)[0]
fetch_loss_and_grads = K.function([generated_image], [loss, grads])
class Evaluator(object):
  def __init__(self):
    self.loss_value = None
    self.grads_values = None
  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value
  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    return grad_values

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
evaluator = Evaluator()

In [39]:
!pip install -U scipy==1.2.0

     |████████████████████████████████| 26.6MB 47.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled scipy-1.3.1


In [16]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'my_result'
iterations = 20

x = preprocess_img(content_path)
x = x.flatten()

for i in range(iterations):
  print('Start at iteration', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
  print('Current loss value:', min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img)
  fname = result_prefix + '_at_iteration_%d.png' % i
  imsave(fname, img)
  print('Image saved as', fname)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start at iteration 0
Current loss value: 1561450400.0
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 16s
Start at iteration 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 733996860.0
Image saved as my_result_at_iteration_1.png
Iteration 1 completed in 7s
Start at iteration 2
Current loss value: 505713280.0
Image saved as my_result_at_iteration_2.png
Iteration 2 completed in 8s
Start at iteration 3
Current loss value: 390873200.0
Image saved as my_result_at_iteration_3.png
Iteration 3 completed in 8s
Start at iteration 4
Current loss value: 333675420.0
Image saved as my_result_at_iteration_4.png
Iteration 4 completed in 8s
Start at iteration 5
Current loss value: 299801700.0
Image saved as my_result_at_iteration_5.png
Iteration 5 completed in 7s
Start at iteration 6
Current loss value: 269744640.0
Image saved as my_result_at_iteration_6.png
Iteration 6 completed in 8s
Start at iteration 7
Current loss value: 248262380.0
Image saved as my_result_at_iteration_7.png
Iteration 7 completed in 7s
Start at iteration 8
Current loss value: 214819360.0
Image saved as my_result_at_iteration_8.png
Iteration 8 completed in 8s
Start at iteration 9
